In [2]:
# relative import ../build/fast_kinematics
import sys
sys.path.append('../build')
import fast_kinematics
import numpy as np
import time
import torch
import pytorch_kinematics as pk

In [19]:
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf",1000,"lbr_iiwa_link_7")

In [20]:
qpos = np.random.rand(7*1000)

In [75]:
start_time = time.time()
for _ in range(100):
  poses = np.array(model.jacobian_mixed_frame(qpos))
  poses = poses.reshape(1000,-1,6).transpose(0,2,1)
end_time = time.time()
print("Time taken for jacobian_mixed_frame: ", end_time - start_time)
# print(poses)

Time taken for jacobian_mixed_frame:  0.6802520751953125


In [76]:
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
N = 1000
d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64

chain = chain.to(dtype=dtype, device=d)
# Jacobian calculation is differentiable
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
# (N,6,7)
start_time = time.time()
for _ in range(100):
  J = chain.jacobian(th)
end_time = time.time()
print("Time: ", end_time - start_time)

Time:  1.3545196056365967
